## Phenotype
### Description
This notebook fetch phenotypes form GEO database and export them as a csv file.

In [ ]:
import GEOparse
import pandas as pd
import re


def match_regex(key, pattern):
    return re.search(pattern, key, re.IGNORECASE) is not None

In [ ]:
gse = GEOparse.get_GEO(geo="GSE89093", destdir="./")

In [12]:
phenotype = pd.DataFrame()

exclude_list = [
    "supplementary_file",
    "data_processing",
    "extract_protocol_ch1",
    "hyb_protocol",
    "contact",
]


def in_exclude_list(key):
    for exclude in exclude_list:
        if match_regex(key, exclude):
            return True
    return False


for gsm_name, gsm in gse.gsms.items():
    basket = {}
    for key, value in gsm.metadata.items():
        if key == "characteristics_ch1":
            cbasket = {}
            for val in value:
                k, v = val.split(": ")
                ktmp = k.replace(" ", "_")
                basket[ktmp] = v
        elif not in_exclude_list(key):
            keytmp = key.replace(" ", "_")
            basket[keytmp] = value[0]
    phenotype = pd.concat(
        [phenotype, pd.DataFrame([basket])], ignore_index=True, axis=0
    )

In [14]:
phenotype.to_csv("phenotype.csv", index=False)